In [1]:
!pip install torch_geometric --quiet

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import os
import json
from torch_geometric.utils import dense_to_sparse
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import OPTICS, DBSCAN
from tqdm.notebook import tqdm
from torch.utils.data import Dataset

# Custom imports
import sys
sys.path.insert(1, "/kaggle/input/second-dataset/dependecies")

import GCN
from Graph import Graph

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEST = False

In [3]:
model_diversity_path = "weights/model_diversity_weights.pth"
model_accuracy_path = "weights/model_accuracy_weights.pth"

In [4]:
input_dim = 8
output_dim = 16
dropout=0

model_diverisity = GCN.GCN(input_dim, output_dim, dropout).to(device)
state_dict = torch.load(model_diversity_path, map_location=device, weights_only=True)
model_diverisity.load_state_dict(state_dict)
model_diverisity.eval()

GCN(
  (gc1): GCNConv(8, 64)
  (gc2): GCNConv(64, 256)
  (gc3): GCNConv(256, 512)
  (gc4): GCNConv(512, 64)
  (residual_proj): Linear(in_features=8, out_features=64, bias=True)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
)

In [5]:
input_dim = 8
output_dim = 1
dropout = 0

model_accuracy = GCN.GCN(input_dim, output_dim, dropout).to(device)
state_dict = torch.load(model_accuracy_path, map_location=device, weights_only=True)
model_accuracy.load_state_dict(state_dict)
model_accuracy.eval()

GCN(
  (gc1): GCNConv(8, 64)
  (gc2): GCNConv(64, 256)
  (gc3): GCNConv(256, 512)
  (gc4): GCNConv(512, 64)
  (residual_proj): Linear(in_features=8, out_features=64, bias=True)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)